### Pinecone Vector DB



In [1]:
## Create your index and apikey from here https://www.pinecone.io/
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
api_key = os.getenv("PINECONE_API_KEY")
api_key[:5]

'pcsk_'

In [7]:
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_key[:5]

'sk-pr'

In [8]:
from pinecone import Pinecone
pc=Pinecone(api_key=api_key)

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small",dimensions=1024,api_key=openai_api_key)

In [10]:
from pinecone import ServerlessSpec

index_name = "udemy-rag"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

/Users/abdullah/Repositories/learning/rag-and-agentic-ai/udemy-rag-course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
index

In [12]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [13]:
vector_store

In [14]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [15]:
vector_store.add_documents(documents=documents)

['4213c407-1e73-47eb-9998-a6cb3a2aaf6e',
 'aa9e2631-8adf-4b0d-90e5-609e2b877a12',
 '7c5f750f-c2fc-42f0-850d-d0ec79c8688e',
 'c70fa765-20de-4244-adc9-7e332b19cf6b',
 '485a056b-96a4-45af-92ac-0b2ad32e03e5',
 '04a46f47-432e-4030-8945-5e4b629d75a4',
 '6e1bf3e1-4eb0-4178-9c13-12a20982fddf',
 '01976dbe-cca9-4925-bc7d-a93b7dcdf49d',
 '6047a656-59b5-4b57-9b41-d05a68c84362',
 '357d89c7-4070-4d99-a3ed-63b497a6725b']

In [16]:
### Query Directly
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [17]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.572732] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [18]:
### Retriever
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.4},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='c70fa765-20de-4244-adc9-7e332b19cf6b', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]